In [1]:
import math as m
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sb
import random
import os
import nltk
from nltk.probability import FreqDist
import re, nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer
import string
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_test = pd.read_csv('./StanceDataset/test_ingrid.csv')
df_train = pd.read_csv('./StanceDataset/train_ingrid.csv')

test_tweet = df_test['Tweet']
train_tweet = df_train['Tweet']

train_sentiment = df_train['Sentiment']

In [ ]:
train_sentiment.value_counts()

In [ ]:
np.mean([len(s.split(" ")) for s in train_tweet])

In [4]:
liste = []

ListeTweet = df_train.Tweet.values.tolist()
ListSentiment = df_train.Sentiment.values.tolist()
dim = len(ListSentiment)
i = 0
#print(dim)

while i < dim:
    tmp = (ListeTweet[i], ListSentiment[i])
    liste.append(tmp)
    i += 1


# liste : Liste(tweet entier, sentiment)
# tweets : Liste(Liste des mots du tweet, sentiment)

tweets = []

for (words, sentiment) in liste:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 4]  #1 pour avoir bcp de mot split
    tweets.append((words_filtered, sentiment))

def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

word_features = get_word_features(get_words_in_tweets(tweets))
#word_features : dictionnaire de mots à partir des tweets

tokens = get_words_in_tweets(tweets)

count1 = Counter(tokens)
print (Counter(count1).most_common(20))
## interessant une fois les mots neutre enlevé

from nltk.corpus import stopwords

filtered = [w for w in tokens if not w in stopwords.words('english')]
count2 = Counter(filtered)
print(Counter(count2).most_common(20))

[('#semst', 2814), ('that', 333), ('have', 240), ('with', 224), ('your', 209), ('this', 184), ('will', 176), ("don't", 164), ('they', 160), ('about', 157), ('what', 152), ('just', 149), ("it's", 134), ('people', 129), ('when', 126), ('like', 124), ('women', 119), ('their', 118), ('@hillaryclinton', 111), ('from', 109)]
[('#semst', 2814), ("don't", 164), ("it's", 134), ('people', 129), ('like', 124), ('women', 119), ('@hillaryclinton', 111), ('hillary', 99), ('want', 96), ('know', 81), ('need', 80), ('right', 77), ("can't", 76), ('would', 75), ('feminists', 74), ('make', 72), ('abortion', 71), ('life', 68), ('think', 66), ('love', 64)]


In [ ]:
from nltk.stem.porter import *

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

stemmer = PorterStemmer()
stemmed = stem_tokens(filtered, stemmer)
count3 = Counter(stemmed)
print(Counter(count3).most_common(20))

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokens,
    lowercase = True,
    stop_words = 'english',
    max_features = 85
)

## autre methode que le vectorisation

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=tokens, stop_words='english')


In [12]:
w = tokens
#nltk.pos_tag(w)
#nltk.help.upenn_tagset()

In [13]:
freq_dist = nltk.FreqDist(w)
#freq_dist.most_common(20)

In [14]:
# This is how the Naive Bayes classifier expects the input
def create_word_features(words):
    useful_words = [word for word in words if word not in stopwords.words("english")]
    my_dict = dict([(word, True) for word in useful_words])
    return my_dict
#For each word, we create a dictionary with all the words and True. Why a dictionary? So that words are not repeated. If a word already exists, it won’t be added to the dictionary.

## METHODE 1

In [ ]:
stop_words=['a', 'able','about','across','after','all','almost','also','am','among','an','and','any','are','as','at','be','because','been','by','did','else','ever','every','for','from','get','got','had','has','have','he','her','hers','him','his','how','however','i','if','in','into','is','it','its','just','least','let','may','me','might','my','of','off','on','or','other','our','own','rather','said','say','says','she','should','since','so','than','that','the','their','them','then','there','these','they','this','tis','to','was','us','was','we','were','what','when','where','while','who','whom','why','will','would','yet','you','your','They','Look','Good','A', 'Able','About','Across','After','All','Almost','Also','Am','Among','An','And','Any','Are','As','At','Be','Because','Been','By','Did','Else','Ever','Every','For','From','Get','Got','Had','Has','Have','He','Her','Hers','Him','His','How','However','I','If','In','Into','Is','It','Its','Just','Least','Let','May','Me','Might','My','Of','Off','On','Or','Other','Our','Own','Rather','Said','Say','Says','She','Should','Since','So','Than','That','The','Their','Them','Then','There','These','They','This','Tis','To','Was','Us','Was','We','Were','What','When','Where','While','Who','Whom','Why','Will','Would','Yet','You','Your','!','@','#','"','$','(','.',')']                                                                                               


#### Extract Feature Vectors

In [ ]:
def getFeatureVector(tweet):
    featureVector = []
    
    #extraction des mots
    words = tweet.split()
    for w in words:
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stop_words or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector

In [ ]:
import re
tweets = []
featureList = []
for i in range(len(df_train)):
    sentiment = df_train['Sentiment'][i]
    tweet = df_train['Tweet'][i]
    ## faire un pré-traitement
    processedTweet = tweet
    featureVector = getFeatureVector(processedTweet)
    featureList.extend(featureVector)
    tweets.append((featureVector, sentiment))
        
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return features


### Remove featureList duplicates
featureList = list(set(featureList))

In [ ]:
#training_set = nltk.classify.util.apply_features(extract_features, tweets)

#NBClassifier = nltk.NaiveBayesClassifier.train(training_set)

#df_train['Sentiment'] = df_train['Tweets'].apply(lambda tweet: NBClassifier.classify(extract_features(getFeatureVector(tweet))))

# METHODE 2
## Sentiwordnet

In [ ]:
train_tweet_pretraitement = [item.lower() for item in train_tweet]
df = pd.DataFrame(train_tweet_pretraitement)
#df

In [ ]:
from nltk.corpus import sentiwordnet as swn


liste = []

ListeTweet = df_train.Tweet.values.tolist()
ListSentiment = df_train.Sentiment.values.tolist()
dim = len(ListSentiment)
i = 0
#print(dim)

while i < dim:
    tmp = (ListeTweet[i], ListSentiment[i])
    liste.append(tmp)
    i += 1

#Liste de tuples (tweet,sentiment)
#print(liste)

# liste : Liste(tweet entier, sentiment)


tweets = []

for (words, sentiment) in liste:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 6]  #1 pour avoir bcp de mot split
    tweets.append((words_filtered, sentiment))

#len(tweets)
#print(tweets)


def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

word_features = get_word_features(get_words_in_tweets(tweets))
# tweets : Liste(mot du tweet, sentiment)
#word_features : Dictionnaire des mots de plus de 6 lettres dans tous les tweets


In [ ]:
### AUTRE METHODE ###

import nltk
#nltk.download('all')

def tokenization(liste):
    token = []
    for (words, sentiment) in liste:
        tmp = nltk.word_tokenize(words)
        token.extend(tmp)
    return token

#token = nltk.word_tokenize(liste)
#tagged = nltk.pos_tag(token)

all_words = tokenization(liste)
tagged = nltk.pos_tag(all_words)


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

lemmatizer = WordNetLemmatizer()

In [ ]:
import pandas as pd
import numpy as np
import nltk
#print("\n\n")
df=pd.read_csv("./StanceDataset/train_ingrid.csv")

#print("Dataset.................")
#print (df)
#print("\n\n")
from nltk.tokenize import word_tokenize
from nltk import pos_tag, pos_tag_sents

In [ ]:
#print("lower..............\n\n")
texts = df['Tweet'].map(lambda Review_Text: Review_Text.lower())#...............converting to   lower case
#print(texts)
df['texts']=texts


In [ ]:
#print("Tokenisation..............\n\n")
token = df.apply(lambda row: nltk.word_tokenize(row['texts']), axis=1)


In [ ]:
df['tokenized_words']=token
#print(df['tokenized_words'])
#print("stopwords removal///////////////////////")
words=df['tokenized_words']

In [ ]:
from nltk.corpus import stopwords
#stop_words = set(stopwords.words('english'))
usertext = token
stop_words=['a', 'able','about','across','after','all','almost','also','am','among','an','and','any','are','as','at','be','because','been','by','did','else','ever','every','for','from','get','got','had','has','have','he','her','hers','him','his','how','however','i','if','in','into','is','it','its','just','least','let','may','me','might','my','of','off','on','or','other','our','own','rather','said','say','says','she','should','since','so','than','that','the','their','them','then','there','these','they','this','tis','to','was','us','was','we','were','what','when','where','while','who','whom','why','will','would','yet','you','your','They','Look','Good','A', 'Able','About','Across','After','All','Almost','Also','Am','Among','An','And','Any','Are','As','At','Be','Because','Been','By','Did','Else','Ever','Every','For','From','Get','Got','Had','Has','Have','He','Her','Hers','Him','His','How','However','I','If','In','Into','Is','It','Its','Just','Least','Let','May','Me','Might','My','Of','Off','On','Or','Other','Our','Own','Rather','Said','Say','Says','She','Should','Since','So','Than','That','The','Their','Them','Then','There','These','They','This','Tis','To','Was','Us','Was','We','Were','What','When','Where','While','Who','Whom','Why','Will','Would','Yet','You','Your','!','@','#','"','$','(','.',')']                                                                                               
clean = usertext.apply(lambda x: [word for word in x if word not in stop_words])
df['clean']=clean
#print(df['clean'])
#print("\n")
#print("Pos_Tagging..............\n\n")
df['tagged_texts'] = df.apply(lambda df:nltk.pos_tag(df['clean']),axis=1)


In [ ]:
#print("\n\n")
#print("Array..............\n\n")
tagged=np.array(df['tagged_texts'])
#print(tagged)

In [ ]:
i=0
dim = len(tagged)
result_sentiment = []


while i < dim:
    
    for word, tag in tagged[i]:
        try:
            ss_set = None
            #print(list(swn.senti_synsets(word))[0])
            if 'NN' in tag and swn.senti_synsets(word):
                ss_set = list(swn.senti_synsets(word))[0]
            elif 'VB' in tag and swn.senti_synsets(word):
                ss_set = list(swn.senti_synsets(word))[0]
            elif 'JJ' in tag and swn.senti_synsets(word):
                 ss_set = list(swn.senti_synsets(word))[0]
            elif 'RB' in tag and swn.senti_synsets(word):
                 ss_set = list(swn.senti_synsets(word))[0]

            if ss_set:
                pos=pos+ss_set.pos_score()
                neg=neg+ss_set.neg_score()
                obj=obj+ss_set.obj_score()
                count+=1

            #print(i)             
        except:
            pass
        
    final_score=pos-neg
    #print(final_score)
    #print("Score sentiment du tweet n°" + i + " : " + final_score)
    print("Score sentiment du tweet n° %d : %s  " % (i, final_score))
    df['final_score']=final_score
    df.to_csv('score_sentiment_save.csv')
    
    if final_score < 0 :
        result_sentiment.append('neg')
    elif final_score > 0 :
        result_sentiment.append('pos')
    elif final_score == 0 :
        result_sentiment.append('other')
    
    print(result_sentiment[i])
    
    #norm_finalscore= round((final_score) / count, 2)
    #print(norm_finalscore)
    #final_sentiment = 'positive' if norm_finalscore >= 0 else 'negative'
    #print(final_sentiment)    
    i+=1
    pos=neg=obj=count=0
    



In [ ]:
label_true=np.array(df_train['Sentiment'])
erreur = 0
dim = len(label_true)

for j in range(dim):
    if label_true[j] != result_sentiment[j]:
        erreur=erreur+1
    j=j+1

In [ ]:
print(dim)
print(erreur)

taux_erreur = (dim-erreur)/dim

print(taux_erreur)

In [ ]:
print("Erreur %s  " % (taux_erreur))